In [267]:
import requests
import csv
import pandas as pd
from bs4 import BeautifulSoup

In [268]:
PAGE_LOAD_TIMEOUT = 5
TIME_SLEEP = 3

DATA_BREAK = 10
PAGE_LIMIT = 50

In [269]:
git_users= ['guilhermefarto', 'RafaelPerbelineSupreme', 'alessandrosilva10', 'nettonucci', 'TobiasEvangelista', 'joao-fernandes-99', 'LucasMPrates', 'femoratolima' ]

In [270]:
def search_gituserinfo(user):
    
    response = requests.get(f'https://api.github.com/users/{user}')
    
    user_details = {}
    user_details['user_id'] = response.json()['id']
    user_details['user_login'] = response.json()['login']
    user_details['user_name'] = response.json()['name']
    user_details['user_avatar_url'] = response.json()['avatar_url']
    user_details['user_location'] = response.json()['location']
    
    return user_details

In [271]:
user_list = {}

In [272]:
for user in git_users:
    user_details = search_gituserinfo(user)
    print(user_details)
    name = user_details['user_login']

    url = f'https://github.com/{name}?tab=repositories'

    repos_gituser = []

    sess = requests.Session()

    url_content = sess.get(url).content

    soup = BeautifulSoup(url_content, 'html.parser')

    all_ul = soup.find_all('div', {'id': 'user-repositories-list'})
    if all_ul is not None:
        all_ul = all_ul[0].find('ul', {'data-filterable-for': 'your-repos-filter'})
        if all_ul is not None:
            all_ul = all_ul.find_all('li')


            for ul in all_ul:
                columns = ul.find('a')
                #print(columns.string)
                repos = columns.string

                repos_gituser.append(repos.strip()) 



                #print (user_details)

            user_details['repos'] = repos_gituser

            user_list[name] = user_details

#print(user_list)
    #print (repos_gituser) 
    #print (user_details)
    #print(all_ul.prettify())



{'user_id': 7007095, 'user_login': 'guilhermefarto', 'user_name': 'Guilherme de Cleva Farto', 'user_avatar_url': 'https://avatars1.githubusercontent.com/u/7007095?v=4', 'user_location': 'São Paulo, Brasil'}
{'user_id': 50852619, 'user_login': 'RafaelPerbelineSupreme', 'user_name': None, 'user_avatar_url': 'https://avatars0.githubusercontent.com/u/50852619?v=4', 'user_location': None}
{'user_id': 29934397, 'user_login': 'alessandrosilva10', 'user_name': None, 'user_avatar_url': 'https://avatars2.githubusercontent.com/u/29934397?v=4', 'user_location': None}
{'user_id': 13320548, 'user_login': 'nettonucci', 'user_name': None, 'user_avatar_url': 'https://avatars3.githubusercontent.com/u/13320548?v=4', 'user_location': None}
{'user_id': 51771359, 'user_login': 'TobiasEvangelista', 'user_name': 'Tobias Evangelista', 'user_avatar_url': 'https://avatars1.githubusercontent.com/u/51771359?v=4', 'user_location': None}
{'user_id': 69994677, 'user_login': 'joao-fernandes-99', 'user_name': None, 'us

In [284]:
def generate_csv(user_list):
    new_user_list = []
    
    for user, repos in user_list.items():
        for repo in repos['repos']:
            new_user_list.append({'git_id': repos['user_id'],'name': repos['user_name'], 'user': user, 'Repo': repo, 'Location': repos['user_location'], 'git_avatar': repos['user_avatar_url']})
    
    dfUsersGit = pd.DataFrame.from_dict(new_user_list, orient='columns')
    dfUsersGit.drop_duplicates(inplace=True)
    dfUsersGit.to_csv(f'Users_github.csv', sep=';', index=False, encoding='iso-8859-1')

In [286]:
generate_csv(user_list)